# Processing excitation data
## Load libraries and data

In [ ]:
# Cell 1: Import Libraries

from chemcompute import chemcompute
from jupyter_jsmol import JsmolView
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Cell 2: Download and view output structure from ChemCompute

job = chemcompute.download(1217373)
view1 = JsmolView.from_file(job, inline=True) 
view1

In [ ]:
# Cell 3: parse output and check values

data = chemcompute.parse(job)
print ("SCF Energy (eV)\n", data.scfenergies)       # SCF energies (eV)
print ("Excitation Energies (cm^-1)\n", data.etenergies) # excitation energies (inverse cm)
print ("Oscillator Strength\n",data.etoscs)     # oscillator strength (unitless)

In [ ]:
# Cell 4: Convert excitation energies to eV

eVenergies=data.etenergies*1.23981*10**-4
print (eVenergies)

# Exercise: Convert energies to nm


# Define function that performs convolution
Definition of convolution for two functions f and g
$$
(f*g)(t):=\int _{-\infty }^{\infty }f(\tau )g(t-\tau )\,d\tau
$$
In this case we first generate a weighted histogram of the excitation energy data and then convolute this function with a Gaussian using the `numpy` function `convolve`.
Note that this function expects energies in nm

In [ ]:
####################################################################################
#                                                                                  #
#    convolute(energy, intensity, fwhm)                                            #
#                                                                                  #
#          Function that sums up gaussian contributions to each point and forms    #
#          into a spectrum (histogram)                                             #
#                                                                                  #
#          inputs:  energy     = List of excitation energies in nm                 #
#                   intensity  = List of oscillator strengths from output          #
#                   fwhm       = The full width of the added gaussian halfway      #
#                                to the maximum of the peak in nm                  # 
#                                                                                  #
#          outputs: ene        = energy values for plotting convoluted spectrum    #
#                   datawithgaussians = Convoluted spectrum                        #
#                                                                                  #
####################################################################################

def convolute(energy,intensity,fwhm):

# width of gaussian (standard deviation) based on user input width
# width is full width at half the maximum (fwhm) of the peak
  sigma = fwhm/(2*np.sqrt(2*np.log(2)))	
  gwidth = sigma*sigma 

# determine range of spectrum to consider
# lowest and highest energies in output file
  mini = min(energy)
  maxi = max(energy)

# increase range of output by 4*sigma so plot does not cut off sharply unless at 0
  mini = max(mini-4*sigma,0)
  maxi = maxi + 4*sigma

# set the number of bins to used in the histogram
  numbins = 1000
  delta = (maxi - mini)/numbins

# form Gaussian to be used in convolution (centered at 0)
  gx=np.arange(-4*sigma,4*sigma,delta)
  gaussian=np.exp(-0.5*(gx*gx)/gwidth)

# Sum and print out total oscillator strength. 
# This should be lower than the total number of electrons based on the TKR sum rules
  total = 0
  for d in range(len(intensity)): 
     total += intensity[d] 

  print("Total Oscillator Strength:", total)
  print(f"Energy Range (nm): {mini:.5}, {maxi:.5}")

# Transform calculated data into a histogram with discritized axis
  calcvals=np.histogram(energy,bins=numbins,range=(mini,maxi), weights=intensity)

# histogram forms energy values as edges of bins, convert to midpoint for plotting as x values
  etemp=calcvals[1]
  ene=[]
  for i in range(len(etemp)-1):
      ene.append((etemp[i]+etemp[i+1])/2)
    
# Convolute calculated data with gaussian 
  datawithgaussians=np.convolve(calcvals[0], gaussian, mode="same")

    
  return ene,datawithgaussians

In [ ]:
# Run convolution function using energies in nm, oscillator strenghts and FWHM of 10 nm for the Gaussian Function

energies,convdata=convolute(nmenergies,data.etoscs,10)

In [ ]:
# Plot broadened spectrum

plt.plot(energies,convdata)              #plot convoluted spectrum
plt.plot(nmenergies,data.etoscs,".")     #plot original oscillator strength vs. energy values
plt.ylabel("Oscillator strength")        #y axis title
plt.xlabel("Energies")                   #x axis title
plt.title('Optical Excitation Spectrum') #plot title
plt.show()

# Exercise: Explore convolution

1. Import data from one of your TD-DFT calculations
2. Convert excitation energies to nm
3. Convolute the spectrum using different values for the FWHM. What is the effect of increasing the FWHM?
4. Plot the convoluted spectrum with the corresponding experimental spectrum. 